# Item 45. 불필요한 객체 생성을 피하라

## java jvm의 객체 재사용, 캐시

1. 문자열
- jvm에서는 하나의 가상 머신에서 같은 문자열 리터럴을 여러 차례 사용해도 실제로는 하나의 String 객체로 생성한다.
2. Integer, Long 박스형 캐싱
- -128 ~ 127까지의 Integer, Long 객체는 미리 생성해 놓고 재사용한다.

In [2]:
val i1 : Int? = 125
val i2 : Int? = 125
println(i1 === i2)

val i3 : Int? = 128
val i4 : Int? = 128
println(i3 === i4)

true
false


## 래핑시 객체 비용은 얼마나 들까?

1. 현대 64비트 jdk에서 객체는 8바이트의 배수만큼 공간을 차지한다.
- 객체의 헤더는 12바이트이며, 8바이트의 배수만큼이 한단위이므로 4바이트 패딩을 가진 16바이트를 차지한다.
- 레퍼런스가 하나있을때마다 4바이트가 필요하다.
e.g) int는 기본적으로 4바이트를 차지한다.(32bit, -2^31~2^31)
e.g) Integer/Int?는 기본패딩 12바이트에 레퍼런스 4바이트가 추가되어 16바이트를 차지한다. 
e.g) 실제 Int? 를 사용할경우 객체 생성 이외에도 참조를 한번 해야하므로 4바이트가  필요하고 패딩으로 4바이트를 쓰므로 총 8바이트가 추가되어 24바이트를 쓰는셈이다. 메모리공간을 6배 차지하는 셈(조각을 포함한것이며 패딩을 제외하면 5배정도 차지한다.)

2. 객체 호출비용
3. 객체 생성시 시간비용
 - 작지만 수억개 생성을 가정하면 무시할수 없는 숫자다

## 객체 재사용하는 방법

### 1. 캐싱 객체 사용하기

- EMPTY_LIST, EMPTY_MAP, EMPTY_SET 과 같이 필요는 하되 변경이 불가능한 객체는 미리 만들어 캐싱해두고 재사용해볼 수 있다
- 도메인 모델 역시 위와같은 특별 케이스는 캐싱해두고 재사용해볼수 있음
- 캐싱시 사용하는 메모리 자체가 부담될 수 있으므로 적절한 캐싱전략, 캐싱시 WeakReference나 SoftReference를 사용하는것도 고려해볼만함
- WeakReference: 가비지컬렉터가 동작할때만 참조를 해제하는 참조
- SoftReference: 메모리가 부족할때 참조를 해제하고 GC에 제거되는 참조

### 2. 무거운 객체를 외부 스코프로 보내기
- 당연한 이야기로 for문안에서 객체를 생성하는것은 비효율적이다.
- forEach, count, map, filter 등의 함수 내에서 객체를 생성하는것도 비효율적이다.
- 정규식도 매번 객체 생성하는게 아니라 톱레벨로 정규표현식을 한번만  생성하는게 더 바람직하다

### 3. 지연 초기화
```
    companion object {
        private val PASSWORD_REGEX by lazy { Regex("^(?=.*[a-z])(?=.*[A-Z])(?=.*\\d)(?=.*[^\\da-zA-Z]).{8,16}\$") }
    }
```
- 코틀린은 손쉽게 지연 초기화를 할 수 있도록 지원한다.
- 다만 지연초기화시엔 속도가 느려질 수 있으므로 적절한 상황에서 사용해야한다.
- lazy는 기본 스레드 세이프하게 Syncronize로 동작한다
  - 다른모드로 사용하고 싶다면 lazy(LazyThreadSafetyMode.NONE)을 사용하면된다.

### 4. 기본 자료형 사용하기

- 코틀린은 모든 자료형 primitive타입을 래핑하지만 코틀린 컴파일러가 자바 바이트코드로 바꿀때, 자바의 primitive타입으로 변경해준다.
- 다만 언박싱은 코틀린 컴파일러의 판단에 따라 알아서 판단하므로 안되는상황도 많다

간단한 비교
|코틀린 자료형|자바의 자료형|
|---|---|
|Int|int|
|Int?|Integer|
|List<Int>|List<Integer>|

- 정말 성능에 민감하다면 배열을 쓰는게 맞고 이때는 IntArray, LongArray등을 사용하면된다.



In [ ]:
// max ?: 때문에 Integer로 박싱되어있음
fun Iterable<Int>.maxOrNull() : Int? {
    var max : Int? = null
    for (i in this) {
        max = if(i > max ?: Int.MIN_VALUE) i else max
    }
    return max
}

// 박싱될 필요가 없으므로 코틀린 컴파일러가 전부 언박싱하여 int로 다룸
fun Iterable<Int>.maxOrNull2() : Int? {
    val iterator = this.iterator()
    if (!iterator.hasNext()) return null
    var max = iterator.next()
    while (iterator.hasNext()) {
        val e = iterator.next()
        if (e > max) max = e
    }
    return max
}